<a href="https://colab.research.google.com/github/AdamHany03/UniversityWork/blob/main/Deep%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
import os

files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
!unzip -q gtsrb-german-traffic-sign.zip -d ./gtsrb
!rm gtsrb-german-traffic-sign.zip

print(os.listdir("./gtsrb"))
print(os.listdir("./gtsrb/Train")[:5])
print(os.listdir("./gtsrb/Test")[:5])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign
License(s): CC0-1.0
 98% 600M/612M [00:05<00:00, 118MB/s]
100% 612M/612M [00:05<00:00, 113MB/s]
replace ./gtsrb/Meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A

A
A
A
['Train.csv', 'Meta.csv', 'Test.csv', 'Test', 'Train', 'test', 'meta', 'train', 'Meta']
['41', '36', '26', '31', '14']
['10277.png', '11862.png', '00237.png', '12048.png', '10850.png']


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import pandas as pd
from tqdm import tqdm
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [62]:
transform_train = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [35]:
train_dataset = datasets.ImageFolder("/content/gtsrb/Train", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
print("Train samples:", len(train_dataset))

Train samples: 39209


In [64]:
import os
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd

class GTSRBTestDataset(Dataset):
    def __init__(self, img_dir, csv_file, transform=None):
        self.img_dir = img_dir
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filename = str(self.data.iloc[idx]["Path"]).split("/")[-1]
        img_name = os.path.join(self.img_dir, filename)
        image = Image.open(img_name).convert("RGB")
        label = int(self.data.iloc[idx]["ClassId"])
        if self.transform:
            image = self.transform(image)
        return image, label

val_dataset = GTSRBTestDataset(
    img_dir="/content/gtsrb/Test",
    csv_file="/content/gtsrb/Test.csv",
    transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
)

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [65]:
model = models.resnet18(pretrained=True)
for param in model.layer4.parameters():
    param.requires_grad = True
model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes))
model = model.to(device)

# 7️⃣ Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [67]:
epochs = 10

for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")

    # 🔹 Train
    model.train()
    train_correct = 0
    train_total = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()
    train_acc = 100 * train_correct / train_total
    print(f"Train Accuracy: {train_acc:.2f}%")


Epoch 1/10


100%|██████████| 1226/1226 [01:18<00:00, 15.71it/s]


Train Accuracy: 66.33%

Epoch 2/10


100%|██████████| 1226/1226 [01:18<00:00, 15.67it/s]


Train Accuracy: 78.26%

Epoch 3/10


...

Train Accuracy: 85.22%

Epoch 9/10


100%|██████████| 1226/1226 [01:15<00:00, 16.24it/s]


Train Accuracy: 85.19%

Epoch 10/10


100%|██████████| 1226/1226 [01:16<00:00, 16.12it/s]

Train Accuracy: 85.45%
